In [31]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

from sklearn.multiclass import OneVsRestClassifier

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extract data samples from raw GDF files

In [32]:
# convert_data()

# Read and split the data

In [40]:
i = 8

x, y = read_file(i)

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 100, test_size = 0.2)

print("final shapes are: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)



final shapes are:  (230, 25, 313) (58, 25, 313) (230,) (58,)


# Extract features from test and train data

In [41]:
train_coeff = featurize(X_train)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
X_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)

test_coeff = featurize(X_test)
X_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)
  

Computing rank from data with rank=None
    Using tolerance 59 (2.2e-16 eps * 25 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 55 (2.2e-16 eps * 25 dim * 9.9e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 62 (2.2e-16 eps * 25 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 50 (2.2e-16 eps * 25 dim * 9.1e+15  max singular value)
    Estimated rank (

# Train model and test

In [43]:
clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
clf.fit(X_train_f, y_train)

print("Accuracy is ", sum(clf.predict(X_train_f) == y_train) / len(X_train))
print(len(X_train))

print("test Accuracy is ", sum(clf.predict(X_test_f) == y_test) / len(X_test))
print(len(X_test))

print("kappa score on train is: ", cohen_kappa_score(clf.predict(X_train_f), y_train))
print("kappa score on test is: ", cohen_kappa_score(clf.predict(X_test_f), y_test))


Accuracy is  0.9782608695652174
230
test Accuracy is  0.8275862068965517
58
kappa score on train is:  0.9709705919474947
kappa score on test is:  0.7654670440760211


# Save the trained model

In [44]:
dump(clf, "model.joblib")
dump(clf, "csp.joblib")

['csp.joblib']